In [1]:
from riptide import *

In [2]:
# Load in models

save riptide models and just read them in here to save time

# Collate reactions that utilize substrates of interest
    saved_rxns = set()
    for rxn in model.reactions:
        for cpd in rxn.reactants:
            if cpd.id in substrates:
                saved_rxns |= set([rxn.id])
    rxn_dict = {}

SyntaxError: invalid syntax (<ipython-input-2-ae19f70a1357>, line 3)

In [13]:
# Virulence algorithm rough draft

from cobra.flux_analysis.variability import *

def virulence_contribution(model, substrates, optimum_fraction=0.85, cytosol='cytosol'):
    # model = COBRA model (REQUIRED)
    # substrates = list or set of metabolite IDs (REQUIRED)
    # optimum_fraction = fraction of optimum to constrain previous objective
    # cytosol = label for intracellular compartment
    
    # Save exchange reactions IDs
    exchanges = set()
    for rxn in model.reactions:
        if len(list(rxn.reactants)) == 0 or len(list(rxn.products)) == 0:
            exchanges |= set([rxn.id])
        
    # Assess essentiality of base model (1% objective flux)
    base_genes = find_essential_genes(model)
    base_rxns = find_essential_reactions(model)
    
    # Handle both intra- and extracellular metabolites
    substrates = [x.split('_')[0] for x in substrates]
    intracellular_cpd = set([x + '_c' for x in substrates])
    extracellular_cpd = set([x + '_e' for x in substrates])
    
    # Create substrate coefficient dictionary
    substrate_coeff_dict = {}
    metabolites = set([x.id for x in model.metabolites])
    for cpd in intracellular_cpd: 
        # check if metabolite is not in the model
        if cpd not in metabolites:
            continue
        else:
            substrate_coeff_dict[model.metabolites.get_by_id(cpd)] = -1.0
    
    # Create virulence metabolite
    virulence_cpd_c = cobra.Metabolite(
    'virulence',
    name='Virulence factors',
    compartment=cytosol)
    substrate_coeff_dict[virulence_cpd_c] = 1.0
        
    with model as m:
        
        # Fix previous objective as a constraint
        prev_obj_val = m.slim_optimize()
        prev_obj_constraint = m.problem.Constraint(m.objective.expression, 
                                                   lb=prev_obj_val*optimum_fraction, ub=prev_obj_val)
        m.add_cons_vars([prev_obj_constraint])
    
        # Add virulence reaction and sink
        virulence_rxn_c = cobra.Reaction('virulence_rxn_c') 
        virulence_rxn_c.name = 'Virulence factor generation'
        virulence_rxn_c.lower_bound = 0.
        virulence_rxn_c.upper_bound = 1000.
        virulence_rxn_c.add_metabolites(substrate_coeff_dict)
        m.add_reactions([virulence_rxn_c])
        m.add_boundary(virulence_cpd_c, type='sink', reaction_id='virulence_DM', ub=1000.)
        
        # Set new objective
        m.objective = m.reactions.virulence_rxn_c

        # Check essentiality, screen against those essential to biomass
        virulence_genes = find_essential_genes(m)
        virulence_genes = virulence_genes.difference(base_genes)
        virulence_genes = set([x.id for x in virulence_genes])
        virulence_rxns = find_essential_reactions(m)
        virulence_rxns = virulence_rxns.difference(base_rxns)
        virulence_substrates = set()
        for rxn in virulence_rxns:
            for cpd in rxn.reactants:
                test = cpd.id.split('_')[0]
                if test not in substrates:
                    virulence_substrates |= set([cpd.id])
        virulence_substrates = virulence_substrates.difference(set(['virulence']))
        # Usually includes several co-factors that aren't super informative
        
    # Look an additonal degree away from the identified nodes, leverage to architecture of the network
    adj_genes, adj_cpds = track_adjacent(model, virulence_rxns, virulence_substrates, virulence_genes)
    
    # Report some stats
    print('Genes directly associated with virulence: ' + str(len(virulence_genes)))
    print('Metabolites directly associated with virulence: ' + str(len(virulence_substrates)) + '\n')
    
    print('Genes indirectly associated with virulence: ' + str(len(adj_genes)))
    print('Metabolites indirectly associated with virulence: ' + str(len(adj_cpds)))
    
    return virulence_genes, virulence_substrates, adj_genes, adj_cpds


def track_adjacent(model, reactions, metabolites, genes, optimum_fraction=0.85):
    
    # Essentiality check for near-optimal virulence production 
    with model as m:
        obj_threshold = m.slim_optimize()
        high_flux_genes = find_essential_genes(m, threshold=obj_threshold * optimum_fraction)
        high_flux_reactions = find_essential_reactions(m, threshold=obj_threshold * optimum_fraction)
        high_flux_substrates = set()
        for rxn in high_flux_reactions:
            for cpd in rxn.reactants:
                if cpd.id not in metabolites:
                    high_flux_substrates |= set([cpd.id])
    
    # Parse model topology for adjacency
    adjacent_substrates = set()
    adjacent_genes = set()
    for cpd1 in metabolites:
        for rxn in model.metabolites.get_by_id(cpd1).reactions:
            if rxn.id not in reactions:
                adjacent_genes |= rxn.genes
                for cpd2 in rxn.reactants:
                    if cpd2.id not in metabolites:
                        adjacent_substrates |= set([cpd2.id])
                
    # Compare to high flux essentiality
    adjacent_substrates = high_flux_substrates.intersection(adjacent_substrates)
    adjacent_substrates = adjacent_substrates.difference(metabolites)
    adjacent_genes = high_flux_genes.intersection(adjacent_genes)
    adjacent_genes = adjacent_genes.difference(genes)
    
    return adjacent_genes, adjacent_substrates
    


In [14]:
iCdJ794 = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_iCdJ794_2019/data/iCdJ794.sbml')
toxin_metabolites = ['cpd00129','cpd00567']

In [15]:
base_genes, base_substrates, adj_genes, adj_cpds = virulence_contribution(iCdJ794, toxin_metabolites)

Genes directly associated with virulence: 8
Metabolites directly associated with virulence: 12

Genes indirectly associated with virulence: 39
Metabolites indirectly associated with virulence: 110


In [19]:
iCdJ794.metabolites.get_by_id(list(adj_cpds)[28]).name

'Phosphatidylglycerol dioctadecanoyl'

In [48]:
iCdJ794.genes.get_by_id(list(base_genes)[7])

Gene identifier,CD630_01120
Name,CD630_01120
Memory address,0x07fcab12c1150
Functional,True
In 7 reaction(s),"rxn00669_c, R01174_2_c, rxn00871_c, R01174_4_c, R01174_5_c, R01174_1_c, R01174_3_c"
